# RetryOutputParser

在某些情况下，通过仅查看输出可以修复所有解析错误，但在其他情况下则不行。一个例子是当输出不仅格式不正确，而且部分不完整时。请参考下面的示例。

In [1]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser, RetryOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [2]:
template = """Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:"""
class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")
        
parser = PydanticOutputParser(pydantic_object=Action)

In [3]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [4]:
prompt_value = prompt.format_prompt(query="who is leo di caprios gf?")

In [5]:
bad_response = '{"action": "search"}'

如果我们尝试直接解析此响应，会出现错误。

In [6]:
parser.parse(bad_response)

OutputParserException: Failed to parse Action from completion {"action": "search"}. Got: 1 validation error for Action
action_input
  field required (type=value_error.missing)

如果我们尝试使用`OutputFixingParser`来修复这个错误，它将会变得困惑 - 即它不知道什么是实际的操作输入。

In [7]:
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [8]:
fix_parser.parse(bad_response)

Action(action='search', action_input='')

相反，我们可以使用RetryOutputParser，该解析器将提示（以及原始输出）传递以再次尝试获得更好的响应。

In [9]:
from langchain.output_parsers import RetryWithErrorOutputParser

In [15]:
retry_parser = RetryWithErrorOutputParser.from_llm(parser=parser, llm=OpenAI(temperature=0))

In [16]:
retry_parser.parse_with_prompt(bad_response, prompt_value)

Action(action='search', action_input='who is leo di caprios gf?')